In [2]:
#import os
#os.getcwd()

'/home/r2sen/private/ECE 176/ECE 176 Project'

In [2]:
PATH = '/home/r2sen/private/ECE 176/ECE 176 Project/'
#import zipfile
#with zipfile.ZipFile(PATH + 'Shift-Net_pytorch-20230314T051253Z-001.zip', 'r') as zip_ref:
    #zip_ref.extractall(PATH)

In [5]:
import shutil
shutil.rmtree(PATH + '__MACOSX')

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import PIL
from torch.utils.data import Dataset
import os
import os.path as osp
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torchvision.transforms as T
from PIL import Image
import numpy as np
import cv2

def get_full_train_list(
    root_dir
):
    data_list = []
    for num in range(1,8145):
        image_name = str(num).zfill(5) + ".jpg"
        data_list.append(osp.join(
            root_dir, image_name)
        )

    return data_list

def get_full_test_list(
    root_dir
):
    data_list = []
    for num in range(1,8041):
        image_name = str(num).zfill(5) + ".jpg"
        data_list.append(osp.join(
            root_dir, image_name)
        )

    return data_list

class S_Cars(Dataset):
    def __init__(
        self,
        data_list
    ):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)
                
    def __getitem__(self, i):
        # input and target images
        in_name = self.data_list[i]
    
        # process the images
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        transf_img = transforms.Compose([
            transforms.ToTensor(),
            normalize,
            transforms.Resize((360,240))
        ])
        
        in_image = Image.open(in_name).convert('RGB')
        
        in_image = transf_img(in_image)

        return in_image

TRAIN_SIZE = 500
VAL_SIZE = 30
TEST_SIZE = 70
full_train_data_list = get_full_train_list(PATH + "Stanford_cars/cars_train")
full_test_data_list = get_full_test_list(PATH + "Stanford_cars/cars_test")

train_data_set = S_Cars(full_train_data_list[: TRAIN_SIZE])
val_data_set = S_Cars(full_train_data_list[TRAIN_SIZE: TRAIN_SIZE + VAL_SIZE])
test_data_set = S_Cars(full_test_data_list[:TEST_SIZE])

print("Training Set Size:", len(train_data_set))
print("Validation Set Size:", len(val_data_set))
print("Test Set Size:", len(test_data_set))


train_loader = torch.utils.data.DataLoader(
    train_data_set, batch_size=32, shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    val_data_set, batch_size=32, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_data_set, batch_size=32, shuffle=False
)

USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

Training Set Size: 500
Validation Set Size: 30
Test Set Size: 70
using device: cuda


In [4]:
import random

def generate_random_mask(height, width, max_size):
    # Randomly choose the center of the mask
    y = random.randint(0, height - 1)
    x = random.randint(0, width - 1)

    # Randomly choose the size of the mask
    size = random.randint(1, max_size)

    # Create the mask tensor
    mask = torch.zeros(3,height, width).to(device)

    # Set the pixels inside the mask to 1
    mask[:,max(0, y - size):min(height, y + size), max(0, x - size):min(width, x + size)] = 1

    return mask.unsqueeze(0)

#mask = generate_random_mask(height, width, max_size)

In [5]:


class PartialConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
        super().__init__()

        # Create a convolution layer with the given parameters
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=bias)

        # Create a mask layer that will keep track of the valid pixels
        self.mask = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.mask.weight.data.fill_(1)

        # Initialize the mask to all ones
        self.mask.requires_grad = False

    def forward(self, x, mask_in=None):
        # If a mask is provided, combine it with the existing mask
        if mask_in is not None:
            mask = self.mask(mask_in)
            self.mask.weight.data = self.mask.weight.data * mask.data

        # Apply the convolution to the input tensor
        raw_out = self.conv(x)

        # Count the number of valid pixels in the mask
        mask_sum = self.mask(x)

        # Normalize the output tensor based on the number of valid pixels
        normalized_out = raw_out / (mask_sum + 1e-8)

        # Apply the mask to the output tensor
        out = normalized_out * self.mask(x)

        return out

class InpaintNet(nn.Module):
    def __init__(self, inp_channels=3, out_channels=3):
        super().__init__()

        # Define the encoder network
        self.encoder = nn.Sequential(
            PartialConv2d(inp_channels, 64, 5, stride=2, padding=2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            PartialConv2d(64, 256, 5, stride=2, padding=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            PartialConv2d(256, 256, 5, stride=2, padding=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

        )

        # Define the decoder network
        self.decoder = nn.Sequential(
            
            PartialConv2d(512, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            PartialConv2d(128, 3, 3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            PartialConv2d(6, 3, 3, stride=1, padding=1),
            nn.BatchNorm2d(6),
            nn.ReLU(),

    )

    def forward(self, x, mask):
        # Apply the mask to the input image
        x = x * mask

        # Encode the masked image
        enc_outputs = []
        enc_outputs.append(x)
        for layer in self.encoder:
            x = layer(x)
            enc_outputs.append(x)
        
        # Reverse the encoder outputs and mask
        enc_outputs = enc_outputs[0: 7:3]
        #mask = mask[::-1]

        up = nn.UpsamplingNearest2d(scale_factor = 2)
        # Decode the masked and encoded image
        for i, layer in enumerate(self.decoder):
            print("x_before:{}".format(x.size()))
            if i%3 == 0:
              x = up(x)
              x = layer(torch.cat([x, enc_outputs[2 - int(i/3)]], dim=1))

        return x


In [6]:
# Create an instance of the inpainting network
def inpaint(image, mask):
  # Convert the image and mask to PyTorch tensors
  
  image = torch.from_numpy(image.transpose((2, 0, 1))).float().unsqueeze(0)
  mask = torch.from_numpy(mask.transpose((2, 0, 1))).float().unsqueeze(0)
  net = InpaintNet()

  # Load the pre-trained weights
  net.load_state_dict(torch.load('inpaint_net_weights.pth', map_location='cpu'))

  # Apply the inpainting network to the image and mask
  output = net(image, mask)

  # Convert the output tensor to a numpy array
  output = output.squeeze().detach().numpy().transpose((1, 2, 0))

  return output


In [7]:
def Evaluate(
    val_loader,
    model,
):
    
    avg_loss = 0
    batch_idx = 0

    model.eval()
    for batch_idx, data in enumerate(val_loader):
        data = data.to(device)
        with torch.no_grad():
            _,_,ht,w = data.size()
            mask = generate_random_mask(ht,w, ht//4)
            score = model(data, mask)

            avg_loss += nn.MSELoss()(score, data)  

    return avg_loss/len(val_loader)

def Train(
    model,
    optim,
    scheduler,
    epochs,
    train_loader,
    val_loader,
    test_loader,
    display_interval = 100
):

    loss_func = nn.MSELoss()
    for i in range(epochs):
        print("Epochs: {}".format(i))
        total_loss = 0
        model.train()
        
        for batch_idx, data in enumerate(train_loader):
            print(data.size())
            data = data.to("cuda:0")
            optim.zero_grad()

            _,_,ht,w = data.size()
            mask = generate_random_mask(ht,w, ht//4)
            score = model(data, mask)
            loss = loss_func(score, data)
            loss.backward()
            optim.step()
            total_loss += loss
            
        
        total_loss /= len(train_loader)
        model.eval()
        avg_val_loss = Evaluate(
            val_loader,
            model
        )
        scheduler.step(total_loss)
        print("Avg Train Loss: {:.4}, Avg Val Loss: {:.4}".format(
            total_loss, avg_val_loss
        ))
    
    return model


from torch.optim.lr_scheduler import ReduceLROnPlateau

def Trainer(model, 
            train_loader,
            val_loader,
            test_loader,
            num_epochs=25, 
            ):
    # define optimizer
    lr = 1e-2
    weight_decay = 1e-5
    optim = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay,
    )
    
    # define learning rate schedule
    scheduler = ReduceLROnPlateau(
        optim, 'min', patience=3,
        min_lr=1e-10, verbose=True
    )
    
    # define loss function

    best_model = Train(
        model,
        optim,
        scheduler,
        num_epochs,
        train_loader,
        val_loader,
        test_loader
    )

    return best_model

In [10]:
model = InpaintNet()
model.to(device)

best_model = Trainer(
    model,
    train_loader,
    val_loader,
    test_loader,
    num_epochs=10
)

Epochs: 0
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:tor

torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([3

torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([3

torch.Size([20, 3, 360, 240])
x_before:torch.Size([20, 256, 45, 30])
x_before:torch.Size([20, 64, 90, 60])
x_before:torch.Size([20, 64, 90, 60])
x_before:torch.Size([20, 64, 90, 60])
x_before:torch.Size([20, 3, 180, 120])
x_before:torch.Size([20, 3, 180, 120])
x_before:torch.Size([20, 3, 180, 120])
x_before:torch.Size([20, 3, 360, 240])
x_before:torch.Size([20, 3, 360, 240])
x_before:torch.Size([30, 256, 45, 30])
x_before:torch.Size([30, 64, 90, 60])
x_before:torch.Size([30, 64, 90, 60])
x_before:torch.Size([30, 64, 90, 60])
x_before:torch.Size([30, 3, 180, 120])
x_before:torch.Size([30, 3, 180, 120])
x_before:torch.Size([30, 3, 180, 120])
x_before:torch.Size([30, 3, 360, 240])
x_before:torch.Size([30, 3, 360, 240])
Avg Train Loss: 2.023, Avg Val Loss: 1.75
Epochs: 4
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])


torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([3

torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([3

torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([20, 3, 360, 240])
x_before:torch.Size([20, 256, 45, 30])
x_before:torch.Size([20, 64, 90, 60])
x_before:torch.Size([20, 64, 90, 60])
x_before:torch.Size([20, 64, 90, 60])
x_before:torch.Size([20, 3, 180, 120])
x_before:torch.Size([2

torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 3, 360, 240])
torch.Size([32, 3, 360, 240])
x_before:torch.Size([32, 256, 45, 30])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 64, 90, 60])
x_before:torch.Size([32, 3, 180, 120])
x_before:torch.Size([3

In [11]:
data = PATH + "Stanford_cars/cars_train/00100.jpg"
    
# process the images
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
transf_img = transforms.Compose([
            transforms.ToTensor(),
            normalize,
            transforms.Resize((360,240))
        ])
        
data = Image.open(data).convert('RGB')
        
data = transf_img(data)[None,:].to(device)

_,_,ht,w = data.size()

mask = generate_random_mask(ht,w, ht//4)
            
score = model(data, mask)

# Convert the output tensor to a numpy array
output = score.squeeze().detach().cpu().numpy().transpose((1, 2, 0))

print(output)

x_before:torch.Size([1, 256, 45, 30])
x_before:torch.Size([1, 64, 90, 60])
x_before:torch.Size([1, 64, 90, 60])
x_before:torch.Size([1, 64, 90, 60])
x_before:torch.Size([1, 3, 180, 120])
x_before:torch.Size([1, 3, 180, 120])
x_before:torch.Size([1, 3, 180, 120])
x_before:torch.Size([1, 3, 360, 240])
x_before:torch.Size([1, 3, 360, 240])
[[[ 0.25370064 -0.16996366 -0.1674066 ]
  [ 0.07177146 -0.183839   -0.17872481]
  [ 0.06340566 -0.2039982  -0.25455344]
  ...
  [-0.14340834 -0.14788057 -0.2310746 ]
  [-0.15562272 -0.13074039 -0.12242739]
  [-0.16351567 -0.08729866 -0.0841136 ]]

 [[ 0.06956276 -0.13933544  0.00151799]
  [-0.08578159 -0.10864893  0.1360305 ]
  [-0.10809651 -0.09997867  0.13410586]
  ...
  [-0.15570115  0.00258154  0.15282783]
  [-0.15928514 -0.03627946  0.15937844]
  [-0.14439447  0.06135446  0.14663759]]

 [[ 0.08834511 -0.19600336 -0.07875486]
  [-0.06425411 -0.12610023  0.06880402]
  [-0.09173547 -0.11764747  0.06954452]
  ...
  [-0.11469173 -0.00646713  0.06929439]

In [18]:
print(output)

[[[ 0.1377022   0.02178742  0.20648146]
  [ 0.10514954  0.17505388  0.13976446]
  [ 0.07234965  0.10395309  0.19436145]
  ...
  [ 0.03320061  0.11962508  0.20224805]
  [ 0.07924954  0.16374156  0.2013998 ]
  [-0.01890031  0.06598755  0.3262776 ]]

 [[ 0.18587516 -0.04081658 -0.03190267]
  [ 0.24968542  0.11121739  0.08860568]
  [ 0.14457953 -0.07644306  0.08077628]
  ...
  [-0.10951841 -0.00205449  0.35027444]
  [-0.06243512  0.06543631  0.23940188]
  [-0.17247143 -0.07966197  0.19013149]]

 [[ 0.2357596   0.00041904  0.12885301]
  [ 0.36927986  0.24507178  0.31225216]
  [ 0.25534046  0.04077709  0.29919344]
  ...
  [-0.17916891  0.04795558  0.3250252 ]
  [-0.09267616  0.13240688  0.18215078]
  [-0.20503004 -0.06166475  0.16120428]]

 ...

 [[ 0.15561591 -0.07721962  0.02286328]
  [ 0.1249164   0.11174791  0.00796901]
  [ 0.09589344  0.11212804  0.12502824]
  ...
  [-0.07370546  0.01836758  0.22339988]
  [-0.04027404  0.02022613  0.13267967]
  [-0.1828407  -0.03044175  0.17765963]]

 [

In [23]:
import cv2
import numpy as np
from matplotlib import pyplot as pyplot
image = output
image = (image - image.min()) / (image.max() - image.min()) #to normalize
image = (image * 255).astype(np.uint8)

print(image[0][0]) #100

image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image[0][0] #[100, 100, 100]

[118 115 120]


error: ignored